# Knickpoint Analyses for Geoscience
-Nick Lewis, Joanmarie Del Vecchio, Chuck Bailey.

Steps
- Get Dem (take user bounds, res.)
- Download geology shp files
- Download WBD GDB
- User can now select what they want to do.
Export files include:
	- Folders for each watershed w name
        - Relief map
            - W/ LSHT highlighted
	    - Slope map (similar but not the same!)
            - LSHT
	    - Knickpoints mapped (maybe adjust delta ksn?  To isolate bedrock reaches.)
		- Profile w trimodal morphology represented (maybe, refer to chuck suggestion)
	- CSVs of knickpoints
		- To be included:
        - Wshed
		- elevation
		- Coords
		- Shapefile geology
		- Nearest roadcut geology that is also within some threshold of elevation from knickpoint.  Do buffer around knickpoint select closest to elevation.
		- Stream cover type (to allow narrowing to bedrock)
		- If BR, fracture data, if AP, bedding
		- If not in AP or BR, Send back a message being like sorry dawg.
		- Depth to mantle @ knickpoint.  Z score to average depth in region.
		- Slope

# Code begins here!

#### Below is a 'code cell'.  This is a block of text that Jupyter and Python use to perform tasks.

#### To run the following code cell and all code cells below, select the cell and press
# <p style="color: aqua;">Shift+Enter.</p>

In [1]:
# This is called a comment.  Denoted by the '#' behind the text, it allows me to better explain code cells while still being a begign line of text.

# First, we import the YML file.  This saves you the work of making your own python environment and installing a bunch of packages.
#!conda env create -f env.yml

In [2]:
# From google drive, let's get the peripheral files our code will use.
#function here that imports shit

In [1]:
# Import packages to our notebook.
# We must do this to use the functions contained inside the packages.
# Some functions have 'as' in the import call.  This is for less typing, and nothing else.
# Google any package name and 'docs' in order to find more detailed explanation of any one package.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import xdem
import os
import rasterio as rio
from rasterio.plot import show
import glob
from concurrent.futures import ThreadPoolExecutor
import requests
import shapely
import time
from tqdm import tqdm

#### Phew!  That was a lot of initial stuff, but no worries.  We can start *really* coding now.

#### Below is a code cell containing our **Constant** Variables, which you may change as you see fit for your particular analyses.

#### Below are 
- Integers
- Lists (denoted by square brackets at the beginning and end, separated by commas [a, b, c, d])
- Strings (contained within quotes 'Like this!')

In [2]:
# Every time you change something within the code cell, run it once more to update the variable.

BOUNDS = [39.0, -80.0, 37.5, -81.5] # In decimal coordinate form.  Format: [N, E, S, W].  Select the smallest area possible in your analyses for maximum resoultion!  
# For now, keep them in Virginia or West VA.

API_KEY = '9086887f175d33aa72eda767f5b1e9cd' # Get your API key from opentopography.org.  Students get higher resolution data!

DEM_DATASET = 'USGS30m' # Options are USGS30m, USGS10m, & USGS1m.  Be warned!  As resolution goes up, available area goes down.

PROJ_TITLE = 'n_lewis' # Create a succinct name with no spaces or leading digits to represent your project file for future exports.

PATH = os.getcwd()

#### If you can't remember what a function does, call the help() function and pass the name of the function you are wondering about inside the colons.

In [3]:
# Downloader
def downloader(bounds:list = BOUNDS, api_key:str = API_KEY, dem_dataset:str = DEM_DATASET, proj_title:str = PROJ_TITLE, path = PATH): # To call a function, we have to define it first.
    """
    This function takes in the user-defined constants and creates a directory.  It then places a geotiff in the directory.

    args: Bounds (list), api_key (str), dem_dataset (str), proj_title (str).

    returns: None.
    """

    # First, we make our directory for project files.
    try:
        os.mkdir(path = f'{path}/{proj_title}')
        print(f'Directory "{path}/{proj_title}" created successfully.') # Tries to make the directory for project files.
        proj_dir = f'{path}/{proj_title}'
    except FileExistsError:
        print(f'Directory "{path}/{proj_title}" already exists.') # If it doesnt work, it'll tell you!
        proj_dir = f'{path}/{proj_title}'
    except Exception as e:
        print(f'An error occurred: {e}')

    # Now to download our dem.
    dic = dict(zip(['north', 'east', 'south', 'west'], bounds)) # Makes a dictionary to let us more easily acess our bounds.
    url = f'https://portal.opentopography.org/API/usgsdem?datasetName={dem_dataset}&south={dic["south"]-0.5}&north={dic["north"]+0.5}&west={dic["west"]-0.5}\
&east={dic["east"]+0.5}&outputFormat=GTiff&API_Key={api_key}' # Creates our URL so we can download.
    print('Querying API...')
    response = requests.get(url) # Saves what the URL spit back at us
    print('URL passed, now waiting for file to write.')
    with open(f'{proj_dir}/gtiff.tiff', 'wb') as file:
        file.write(response.content) # Writes the response to our project file
    print('File Written.')

downloader() # Here, we call the function.

Directory "/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis" already exists.
Querying API...
URL passed, now waiting for file to write.
File Written.


#### Next we will clip out each watershed in your selected area.

In [6]:
# Wshed Clipper
def wshed_clipper(proj_title = PROJ_TITLE, bounds = BOUNDS, path = PATH):
    """
    
    """
    # PLACEHOLDER - Line to download wsheds csv
    # df = pd.read_csv(f'{os.getcwd}/{proj_title}/wsheds.csv') # We read in our wshed data
    print('Loading data...')
    df = pd.read_csv('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/data/wbd_scale/WBDHU10.csv') 
    gdf = gpd.GeoDataFrame(data=df, crs=rio.CRS.from_epsg(4269), geometry=[shapely.wkt.loads(x) for x in df['geometry']]) # We turn it into a geodataframe to do geospatial stuff to.
    clipped = gpd.GeoDataFrame(gdf.cx[bounds[3]:bounds[1], bounds[2]:bounds[0]]) # Clip it to the user's bounds.

    # Now that we have a GDF (GeoDataFrame) with the user's chosen watersheds and the larger dem, lets export DEMs of each watershed.
    try:
        os.mkdir(path = f'{path}/{proj_title}/wshed_dems') # Makes a directory to store these new dems.
        print(f'Directory "{path}/{proj_title}/wshed_dems" created successfully.') # Tries to make the directory for project files.
    except FileExistsError:
        print(f'Directory "{path}/{proj_title}/wshed_dems" already exists.') # If it doesnt work, it'll tell you!
    except Exception as e:
        print(f'An error occurred: {e}')
    
    # We have our directory, lets populate it.
    print('Writing new DEMs...')
    with rio.open(f'{path}/{proj_title}/gtiff.tiff') as src: # Loads in our larger df.

        total_tasks=len(clipped)
        with tqdm(total=total_tasks, desc='Progress') as pbar: # This is for the fancy progress bar.

            for idx, row in clipped.iterrows(): # Iterate through the watershed dataframe (for each row, do something.)
                name = row['name'].replace(' ', '_') # Replaces spaces with underscores and saves the name to a variable.

                geom = row['geometry'] # Saves our geometry to a variable.

                out_image, out_transform = rio.mask.mask(src, geom.geoms, crop=True, nodata=-99999) # Clips the DEM to our watershed's shape.  Sets nodata value to LSDTopyTools standard.

                # Below is metadata stuff, dont stress about this.
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                    })

                out_path = f'{path}/{proj_title}/wshed_dems/{name}.tiff' # Create a pathname.
                with rio.open(out_path, "w", **out_meta) as file:
                    file.write(out_image) # Write to file.
                pbar.update(1)

# Call our function, let it do the work.           
wshed_clipper()

Loading data...
Directory "/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/wshed_dems" already exists.
Writing new DEMs...


Progress: 100%|██████████| 67/67 [00:06<00:00, 10.02it/s]


In [ ]:
for x in glob.glob('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/wshed_dems/*.tiff'):
    with rio.open(x) as src:
        show(src)

In [1]:
# Relief & LSHT toolbox


In [ ]:
# KSN toolbox

In [ ]:
# Mapper

In [ ]:
# DF Maker

In [ ]:
# Exporter

In [5]:
import time
from tqdm import tqdm

total_tasks = 100
with tqdm(total=total_tasks, desc="Performing tasks") as pbar:
    for i in range(total_tasks):
        # Simulate some work
        time.sleep(0.05)
        pbar.update(1) # Increment the progress bar by 1

Performing tasks: 100%|██████████| 100/100 [00:05<00:00, 19.90it/s]


In [ ]:
# Kept to show how wshed dataset was narrowed.  U10 scale and only regions in the appalachians.
#apps = gdf.cx[-87:-58, 32:49]
#apps.to_csv('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/data/wbd_scale/WBDHU10.csv')